## <font color=green> *FACTS GMD:* <font color=black> Figure 6 </font>
#### <font color=blue> *VARIANCE DECOMPOSITION* PLOT </font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr
#
import warnings
import matplotlib
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)
#
# colrs = 'rcbgmyrkcbgm'
colrs = 'rcgmbyrkcbgm'

### <font color=blue> List of **Modules** and **Data** to be used </font>

In [2]:
# Load FACTS workflow data as a dataframe.
#
df_cpl_G = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_coupling.txt',comment = '#')
df_cpl_L = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_coupling.local.txt',comment = '#')
#
df_ark_G = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14-221217.txt',comment = '#')
df_ark_L = pd.read_fwf('../001_GMD_IP_Modules_Data/ModuleTest_DataFile_List_ar5k14.local-221217.txt',comment = '#')
#
df_cpl_L
# df_ark_G

,Component,Module,sub_Module,DataFile
0,TE,tlm,sterodynamics,ocean.tlm.sterodynamics_localsl.nc
1,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_localsl.nc
2,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_localsl.nc
3,GrIS,bamber19,icesheets,bamber19.bamber19.icesheets_GIS_localsl.nc
4,GrIS,FittedISMIP,GrIS,GrIS1f.FittedISMIP.GrIS_GIS_localsl.nc
5,GrIS,emulandice,GrIS,emuGrIS.emulandice.GrIS_localsl.nc
6,AIS,bamber19,icesheets,bamber19.bamber19.icesheets_AIS_localsl.nc
7,AIS,deconto21,AIS,deconto21.deconto21.AIS_AIS_localsl.nc
8,AIS,emulandice,AIS,emuAIS.emulandice.AIS_localsl.nc
9,AIS,larmip,AIS,larmip.larmip.AIS_localsl.nc


#### <font color=black> *Define* Functions </font>

##### <font color=blue> Input </font> <mark> Data </mark>

In [3]:
def IP_nc(DF,WF,SCENARIO,EXPDATE):
    #  
    a = [];     ccomp = []
    #
    for scenario in SCENARIO:
        for wf in WF: 
            #
            # Pick Components for each workflow.
            COMP = wf.split("-")[0]; MODULE = wf.split("-")[1]; 
            val = DF.index[ (DF['Component'] == COMP) & (DF['Module'] == MODULE) ].values[0]
            SUB_MODULE = DF["sub_Module"][val];  DATAFILE   = DF["DataFile"][val];
            #       
            # Skip if data is absent.
            if DF["DataFile"][val] == 'XXX':
                continue 
            #
            # Pick the data files & Import nc file to dataframe. 
            if SCENARIO[0][:3] == 'ssp':
                dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
                dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + DF["DataFile"][val]
            elif SCENARIO[0][:3] == 'rcp':
                dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/ar5k14.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
                dataFILE    = 'ar5k14.{arg1}.'.format(arg1=scenario) + DF["DataFile"][val]
            d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
            #print( "DataFILE ==> "+str(DF["DataFile"][val])+"\n"+ str(d_nc.dims) + "\n" + str(d_nc.data_vars)+ "\n" + str(d_nc.coords)+"\n"+"-------------"+"\n"+"\n" )
            #
            # Index for time.
            ST = 2020 ; EN = 2100
            YindST = np.where(d_nc["years"].values == ST)[0][0];   YindEN = np.where(d_nc["years"].values == EN)[0][0]
            # Save data into a new variable.
            ccomp.append(COMP)
            b = d_nc.sea_level_change[:,YindST:YindEN+1,0].values
            a.append(b[None,:] )
    #stack all at once
    sampsloccomponents = np.vstack(a);  sampsloccomponents = np.transpose(sampsloccomponents,(1,0,2))
    yrs=d_nc.years[YindST:YindEN+1].values
    #
    return sampsloccomponents, ccomp, yrs; 

##### <font color=blue> Compute </font> <mark> Variance </mark>

In [4]:
def varV_varF(sampsloccomponents,ccomp):
    #
    varV = []; varF = [];  valIND = 0
    denom=np.var(np.sum(sampsloccomponents[:,:,:],axis = 1),axis=0)
    for co in ccomp:
        # u = np.sum(sampsloccomponents[:,0:valIND+1,:],axis = 1)
        u = sampsloccomponents[:,valIND,:]
        VAR_V=(np.var(u,axis=0))/1e6
        VAR_F=(np.var(u,axis=0))/denom
        varV.append(VAR_V)
        varF.append(VAR_F)
        valIND += 1    
    return varV, varF

# <font color=green> PLOT==> **Coupling** *(Global)* ssp/wf</font>

In [5]:
EXPDATE  = 221217
wf1f_cpl = ['AIS-ipccar5'   , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm']
wf2f_cpl = ['AIS-larmip'    , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm']
wf3f_cpl = ['AIS-deconto21' , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm']
wf4_cpl  = ['AIS-bamber19'  , 'GrIS-bamber19'    ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm']
#
workflow = ['wf1f_cpl', 'wf2f_cpl', 'wf3f_cpl','wf4_cpl']
ssp      = ['ssp126','ssp585'] 
#
# df = df_cpl_G;
# #
# #
# plt.subplots(figsize=(10, 10));
# #
# # ---------   SSP126
# #
# # Subplot will alternate between ssp
# subPLT = 0

# for wf in enumerate(workflow):
#     for ss in enumerate(ssp):
#         [sampsloccomponents, ccomp, yrs] = IP_nc(df,eval(f'{wf[1]}'),[ss[1]],EXPDATE)
#         [varV, varF] = varV_varF(sampsloccomponents,ccomp)
#         # Plot Fraction of variance.
#         ax = plt.subplot(4, 2, subPLT+1); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4], labels=ccomp, colors=colrs);
#         #
#         if wf[1] == 'wf1f_cpl' & ss[1] == 'ssp126':  ax.set_title('SSP1-2.6'); ax.set_ylim([0, 1]);
#         if wf[1] == 'wf1f_cpl' & ss[1] == 'ssp585':  ax.set_title('SSP5-8.5'); ax.set_ylim([0, 1]);
#         #
#         if wf[1] == 'wf2f_cpl' & ss[1] == 'ssp126':  ax.set_title('SSP1-2.6'); ax.set_ylim([0, 1.1]); ax.axhline(y=1,color='black'); ax.legend(loc='lower right'); 
#         if wf[1] == 'wf2f_cpl' & ss[1] == 'ssp585':  ax.set_title('SSP5-8.5'); ax.set_ylim([0, 1.1]); ax.axhline(y=1,color='black')
#         #
#         if wf[1] == 'wf3f_cpl' & ss[1] == 'ssp126':  ax.set_title('SSP1-2.6'); ax.set_ylim([0, 1]);
#         if wf[1] == 'wf3f_cpl' & ss[1] == 'ssp585':  ax.set_title('SSP5-8.5'); ax.set_ylim([0, 1]);
#         #
#         if wf[1] == 'wf4_cpl'  & ss[1] == 'ssp126':  ax.set_title('SSP1-2.6'); ax.set_ylim([0, 1.6]); ax.axhline(y=1,color='black')
#         if wf[1] == 'wf4_cpl'  & ss[1] == 'ssp585':  ax.set_title('SSP5-8.5'); ax.set_ylim([0, 1.6]);ax.axhline(y=1,color='black')
#         #
#         if wf[1] != 'wf4_cpl': ax.set_xlabel(''); ax.xaxis.set_ticklabels([])
#         #
#         #
#         ax.set_xlim([2020, 2100]); 
#         ax.set_ylabel('Fraction of variance');
#         ax.text(.07, .1, 'Workflow '+wf[1][2]+'f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
#         ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
# # ---------
# #
# plt.subplots_adjust(wspace=0.35, hspace=0.4)
# #
# #
# # plt.savefig("Fig6_G_quad.pdf", format="pdf", bbox_inches="tight")
# plt.show()

In [ ]:
for wf in enumerate(workflow):
    print(wf)
    # for ss in enumerate(ssp):
    #     if wf[1] == 'wf1f_cpl' & ss[1] == 'ssp126':  
    #         print('LOOOP WOrks')
    #     else:
    #         print('Fail') 


# <font color=green> PLOT==> **Coupling** *(Local)* ssp/wf</font>

In [ ]:
EXPDATE  = 221217
wf1f_cpl = ['WAIS-ipccar5', 'EAIS-ipccar5' , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm' , 'VLM-kopp14' ]
wf2f_cpl = ['AIS-larmip'    , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm' , 'VLM-kopp14' ]
wf3f_cpl = ['AIS-deconto21' , 'GrIS-FittedISMIP' ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm' , 'VLM-kopp14' ]
wf4_cpl  = ['AIS-bamber19'  , 'GrIS-bamber19'    ,  'Glaciers-ipccar5-GMIP2'  ,  'LWS-ssp' , 'TE-tlm' , 'VLM-kopp14' ]
#
#
plt.subplots(figsize=(10, 10));
#
#
# ---------   SSP126
#
# For wf1f, there is no AIS.nc, so add WAIS and EAIS.
[slcDUM, ccompDUM, yrs] = IP_nc(df_cpl_L,wf1f_cpl,['ssp126'],EXPDATE);
sampsloccomponents      = np.concatenate((slcDUM[:, :1, :] + slcDUM[:, 1:2, :], slcDUM[:, 2:, :]), axis=1)
ccomp                   = ['AIS', 'GrIS' ,  'Glaciers'  ,  'LWS' , 'TE' , 'VLM' ]
#
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax = plt.subplot(4, 2, 1); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4], varF[5],labels=ccomp, colors=colrs);
ax.set_title('New York: SSP1-2.6'); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1]);
ax.xaxis.set_ticklabels([])
ax.text(.7, .1, 'Workflow 1f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
[sampsloccomponents, ccomp, yrs] = IP_nc(df_cpl_L,wf2f_cpl,['ssp126'],EXPDATE);
ccomp[4] = 'SD' # Set lengend label
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax = plt.subplot(4, 2, 3); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1.05]);
ax.xaxis.set_ticklabels([])
# ax.legend(loc='lower right');
ax.text(.7, .1, 'Workflow 2f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
[sampsloccomponents, ccomp, yrs] = IP_nc(df_cpl_L,wf3f_cpl,['ssp126'],EXPDATE);
sampsloccomponents = slc_cpl_L_26_w3;      ccomp = ccomp_cpl_L_26_w3; 
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax=plt.subplot(4, 2, 5); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1]);
ax.xaxis.set_ticklabels([])
ax.text(.7, .1, 'Workflow 3f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
sampsloccomponents = slc_cpl_L_26_w4;      ccomp = ccomp_cpl_L_26_w4;
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax=plt.subplot(4, 2, 7); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
# ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1.3]); ax.axhline(y=1,color='black')
ax.set_xlim([2020, 2100]);ax.set_ylim([0, ax.get_ylim()[1]+0.02]); ax.axhline(y=1,color='black')
#==>
ax.text(.7, .1, 'Workflow 4', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
# ---------   SSP585
#
#
# For wf1f, there is no AIS.nc, so add WAIS and EAIS.
[slcDUM, ccompDUM, yrs] = IP_nc(df_cpl_L,wf1f_cpl,['ssp585'],EXPDATE);
sampsloccomponents      = np.concatenate((slcDUM[:, :1, :] + slcDUM[:, 1:2, :], slcDUM[:, 2:, :]), axis=1)
ccomp                   = ['AIS', 'GrIS' ,  'Glaciers'  ,  'LWS' , 'TE' , 'VLM' ]
#
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax = plt.subplot(4, 2, 2); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title('New York: SSP5-8.5'); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1]);
ax.xaxis.set_ticklabels([])
ax.legend(loc='lower left');
ax.text(.7, .1, 'Workflow 1f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
sampsloccomponents = slc_cpl_L_85_w2;      ccomp = ccomp_cpl_L_85_w2;
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax=plt.subplot(4, 2, 4); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1.05]); ax.axhline(y=1,color='black')
# ax.set_xlim([2020, 2100]);ax.set_ylim([0, ax.get_ylim()[1]+0.02]); ax.axhline(y=1,color='black')
#==>
ax.xaxis.set_ticklabels([])
ax.text(.7, .1, 'Workflow 2f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
sampsloccomponents = slc_cpl_L_85_w3;      ccomp = ccomp_cpl_L_85_w3;
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax=plt.subplot(4, 2, 6); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1]);
ax.xaxis.set_ticklabels([])
ax.text(.7, .1, 'Workflow 3f', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
#
sampsloccomponents = slc_cpl_L_85_w4;      ccomp = ccomp_cpl_L_85_w4;
[varV, varF] = varV_varF(sampsloccomponents,ccomp)
# Fraction of variance.
ax=plt.subplot(4, 2, 8); ax.stackplot(yrs,varF[0],varF[1], varF[2], varF[3], varF[4],  varF[5],labels=ccomp, colors=colrs);
ax.set_title(''); ax.set_xlabel(''); ax.set_ylabel('Fraction of variance');
# ax.set_xlim([2020, 2100]); ax.set_ylim([0, 1.3]); ax.axhline(y=1,color='black')
ax.set_xlim([2020, 2100]);ax.set_ylim([0, ax.get_ylim()[1]+0.02]); ax.axhline(y=1,color='black')
#==>
ax.text(.7, .1, 'Workflow 4', horizontalalignment='left',verticalalignment='top',weight='bold',fontsize=10,transform=ax.transAxes)
ax.tick_params(axis='both', labelsize=10); ax.yaxis.set_ticks_position('both')
#
# ---------

#
plt.subplots_adjust(wspace=0.35, hspace=0.4)
#
#
plt.savefig("Fig6_L_quad.pdf", format="pdf", bbox_inches="tight")
plt.show()